In [1]:
%matplotlib inline
import matplotlib as plt
import numpy as np
import scipy.optimize as opt

In [2]:
#   R|RxR L|RxR R|RxL L|RxL R|LxL L|LxL
D = [841, 115,  113,  54,   1,    7]
D = np.array(D)

In [3]:
def compute_T(ρ, α, β):
    """Generate T - column DD in Table 1
    
    From Appendix 3:    
    T is a 3 x 2 matrix of p(Ht | Ht x Ht) entries - 
    probability of a truly H child from a mating where parents are truly H
    """
    return np.array([
        # R           L
        [0.5 + ρ + α, 1 - (0.5 + ρ + α) ],# RxR
        [0.5 + ρ + β, 1 - (0.5 + ρ + β) ],# RxL
        [0.5 + ρ - α, 1 - (0.5 + ρ - α) ] # LxL
    ])

In [4]:
def compute_t(ρ, α, β):
    """Generate t - eq .3 - the true incidence of left-handedness
    
    From main text:
    F_DR is the final equilibrium frequency of right-handers with allele D fixed
    We need F_DL = 1 - F_DR
    """
    if β == 0:        
        FDR = (1 - 2 * α + 2 * ρ) / (2 * (1 - 2 * α)) # eq 3a
    else:
        Δ = 4 * α * α - 4 * α + 4 * β * β + 1 + 8 * β * ρ
        FDR = (2 * α + 2 * β - 1 + np.sqrt(Δ)) / (4 * β) # eq 3    
    return 1 - FDR

In [5]:
def compute_mp_mo(D, t):    
    mp = (D[2] + D[3] + 2*D[4] + 2*D[5]) / (2*D.sum())
    mo = (D[1] + D[3] + D[5]) / D.sum()
    return mp, mo

In [6]:
def compute_P(mp, t, u, v):
    if mp > t:
        P = np.array([
            [1, 0 ,0],
            [u, 1-u, 0],
            [u*u, 2*u*(1-u), (1-u)*(1-u)]
        ])
    else:
        P = np.array([
            [(1-v)*(1-v), 2*v*(1-v), v*v],
            [0, 1-v, v],
            [0, 0, 1]
        ])
    return P

In [7]:
def compute_O(mo, t, w, x):
    """Appendix 3"""
    if mo > t:
        O = np.array([
            [1-w, w],
            [0, 1]
        ])
    else:
        O = np.array([
            [1, 0], 
            [x, 1-x]
        ])
    return O

In [8]:
def compute_M(T, P, O):    
    M = P @ T @ O
    return M

In [9]:
def loglik(θ, D):
    ρ, α, β, u, v, w, x = θ
    T = compute_T(ρ, α, β)
    t = compute_t(ρ, α, β)
    mp, mo = compute_mp_mo(D, t)
    P = compute_P(mp, t, u, v)
    O = compute_O(mo, t, w, x)
    M = compute_M(T, P, O)
    logM = np.log(M)
    S = (
    D[0] * logM[0,0] + # R|RxR
    D[1] * logM[0,1] + # L|RxR
    D[2] * logM[1,0] + # R|RxL
    D[3] * logM[1,1] + # L|RxL
    D[4] * logM[2,0] + # R|LxL
    D[5] * logM[2,1] ) # L|LxL
    return S 

def negloglik(θ, D):
    return -loglik(θ, D)

In [10]:
θ = 0.148, 0.012, 0.267, 0.05, 0.05, 0.05, 0.05
θ0 = 0.1, 0.01, 0.1, 0.05, 0.05, 0.05, 0.05

In [11]:
negloglik(θ, D)

585.1481382973433

In [15]:
res = opt.minimize(
    negloglik, [θ], args=(D,), options=dict(maxiter=1000),
    method="Nelder-Mead", bounds=[(0,1)]*7)
res

 final_simplex: (array([[0.00183845, 0.47587983, 0.        , 0.        , 0.18689672,
        0.04369117, 0.40433506],
       [0.00184392, 0.47586151, 0.        , 0.        , 0.1868712 ,
        0.0437649 , 0.40428661],
       [0.00184314, 0.47586132, 0.        , 0.        , 0.18686871,
        0.04375977, 0.40429008],
       [0.00183949, 0.47588618, 0.        , 0.        , 0.18689912,
        0.04369242, 0.40433928],
       [0.00184322, 0.4758628 , 0.        , 0.        , 0.18687999,
        0.04374908, 0.40429382],
       [0.00183993, 0.47587699, 0.        , 0.        , 0.18689841,
        0.04370348, 0.40432548],
       [0.00184308, 0.47586339, 0.        , 0.        , 0.18686762,
        0.04376669, 0.40428829],
       [0.00183855, 0.47585609, 0.        , 0.        , 0.1868846 ,
        0.04367592, 0.40432688]]), array([461.38488519, 461.3848852 , 461.38488521, 461.38488522,
       461.38488522, 461.38488522, 461.38488524, 461.38488527]))
           fun: 461.384885188322
       messa

In [16]:
res.x, res.fun

(array([0.00183845, 0.47587983, 0.        , 0.        , 0.18689672,
        0.04369117, 0.40433506]),
 461.384885188322)